In [7]:
f = open('./자가증식텍스트.txt', encoding='utf-8')

sentences = list()
golden = list()
for line in f.read().splitlines():
    sentences.append(line.split()[0])
    golden.append(int(line.split()[1]))
f.close()

f = open('./yonsei_sendData.txt', encoding='utf-8')

origin_sentences = list()
origin_tag = list()

for line in f.read().splitlines():
    origin_sentences.append(line.split(',')[2].replace(' ',''))
    if line.split(',')[0] == '1.0':
        origin_tag.append(1)
    else:
        origin_tag.append(0)
f.close()

print(len(origin_sentences))

126


In [0]:
def get_score(predict):
    tp,tn,fp,fn = 0,0,0,0
    for i in range(len(golden)):
        if golden[i] == 1 and predict[i] == 1:
            tp += 1
        elif golden[i] == 1 and predict[i] == 0:
            fn += 1
        elif golden[i] == 0 and predict[i] == 0:
            tn += 1
        elif golden[i] == 0 and predict[i] == 1:
            fp += 1
    acc = float(tp+tn)/len(golden)        
    precision = float(tp)/(tp+fp)
    recall = float(tp)/(tp+fn)
    fscore = (precision*recall)/(precision+recall)*2
    return fscore, precision, recall, acc

In [9]:
f = open('./blacklist1.txt',encoding='utf-8')

black_list = list()
for line in f.read().splitlines():
    black_list.append(line.split()[0])
f.close()

black_list = list(set(black_list))
predict_blacklist = list()
for sentence in sentences:
    black_flag = 0
    for black_word in black_list:
        if black_word in sentence:
            black_flag = 1
            break
    if black_flag == 1:
        predict_blacklist.append(1)
    else:
        predict_blacklist.append(0)
fscore1,a,b,c = get_score(predict_blacklist)
print(fscore1,a,b,c)

0.7133757961783439 0.7 0.7272727272727273 0.85


In [0]:
import unicodedata
import string
import re
import random
import time
import math

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
import numpy as np

In [0]:
USE_CUDA = True

In [0]:
SOS_token = 0
EOS_token = 1
UNK_token = 2

MAX_LENGTH = 34

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS", 2: "UNK"}
        self.n_words = 3 # Count SOS and EOS
      
    def index_words(self, sentence):
        for word in sentence.split(' '):
            self.index_word(word)

    def index_word(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

input_lang = Lang('src')
output_lang = Lang('targ')

# src_f = open('./data/original_src-train.txt','r')
# targ_f = open('./data/original_targ-train.txt','r')

src_f = open('./non비속어_balanced.txt','r',encoding='UTF8')
# targ_f = open('./original_targ-train.txt','r')

src, targ = list(),list()

for line in src_f.read().splitlines():
    src_element = line.split('\t')[0].replace(" ", "")
    if len(src_element) > 20:
        src_element = src_element[len(src_element)-20:len(src_element)]
        src_element = list(src_element)
        src_element = ' '.join(src_element)
        src.append(src_element)
    else:
        src_element = list(src_element)
        src_element = ' '.join(src_element)
        src.append(src_element)
    targ.append(line.split('\t')[1])
    input_lang.index_words(src_element)
    output_lang.index_words(line.split('\t')[1])
    
pairs = list()
for i in range(0,len(src)):
    pairs.append([src[i]] + [targ[i]])

tmp_dict = {}

for element in input_lang.word2count.items():
  if element[1] > 1:
    tmp_dict[element[0]] = element[1]

input_lang.word2count = tmp_dict    

In [0]:
def indexes_from_sentence(lang, sentence):
    indexes = list()
    for word in sentence.split(' '):
      if word in lang.word2count:
        indexes.append(lang.word2index[word])
      else:
        indexes.append(2)
    return indexes

def variable_from_sentence(lang, sentence):
    indexes = indexes_from_sentence(lang, sentence)  
    indexes.append(EOS_token)
    var = Variable(torch.LongTensor(indexes).view(-1,1))
#   print('var = ',var)
    if USE_CUDA: var = var.cuda()
    return var

def variables_from_pair(pair):
    input_variable = variable_from_sentence(input_lang, pair[0])
    target_variable = variable_from_sentence(output_lang, pair[1])
    return (input_variable, target_variable)


In [0]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, n_layers=1):
        super(EncoderRNN, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers)
        
    def forward(self, word_inputs, hidden):
        # Note: we run this all at once (over the whole input sequence)
        seq_len = len(word_inputs)
        embedded = self.embedding(word_inputs).view(seq_len, 1, -1)
        output, hidden = self.gru(embedded, hidden)
        return output, hidden

    def init_hidden(self):
        hidden = Variable(torch.zeros(self.n_layers, 1, self.hidden_size))
        if USE_CUDA: hidden = hidden.cuda()
        return hidden

In [0]:
class BahdanauAttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, n_layers=1, dropout_p=0.1):
        super(AttnDecoderRNN, self).__init__()
        
        # Define parameters
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout_p = dropout_p
        self.max_length = max_length
        
        # Define layers
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.dropout = nn.Dropout(dropout_p)
        self.attn = GeneralAttn(hidden_size)
        self.gru = nn.GRU(hidden_size * 2, hidden_size, n_layers, dropout=dropout_p)
        self.out = nn.Linear(hidden_size, output_size)
    
    def forward(self, word_input, last_hidden, encoder_outputs):
        # Note that we will only be running forward for a single decoder time step, but will use all encoder outputs
        
        # Get the embedding of the current input word (last output word)
        word_embedded = self.embedding(word_input).view(1, 1, -1) # S=1 x B x N
        word_embedded = self.dropout(word_embedded)
        
        # Calculate attention weights and apply to encoder outputs
        attn_weights = self.attn(last_hidden[-1], encoder_outputs)
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1)) # B x 1 x N
        
        # Combine embedded input word and attended context, run through RNN
        rnn_input = torch.cat((word_embedded, context), 2)
        output, hidden = self.gru(rnn_input, last_hidden)
        
        # Final output layer
        output = output.squeeze(0) # B x N
        output = F.log_softmax(self.out(torch.cat((output, context), 1)))
        
        # Return final output, hidden state, and attention weights (for visualization)
        return output, hidden, attn_weights

In [0]:
class Attn(nn.Module):
    def __init__(self, method, hidden_size, max_length=MAX_LENGTH):
        super(Attn, self).__init__()
        
        self.method = method
        self.hidden_size = hidden_size
        
        if self.method == 'general':
            self.attn = nn.Linear(self.hidden_size, hidden_size)

        elif self.method == 'concat':
            self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
            self.other = nn.Parameter(torch.FloatTensor(1, hidden_size))

    def forward(self, hidden, encoder_outputs):
        seq_len = len(encoder_outputs)

        # Create variable to store attention energies
        attn_energies = Variable(torch.zeros(seq_len)) # B x 1 x S
        if USE_CUDA: attn_energies = attn_energies.cuda()

        # Calculate energies for each encoder output
        for i in range(seq_len):
            attn_energies[i] = self.score(hidden, encoder_outputs[i])

        # Normalize energies to weights in range 0 to 1, resize to 1 x 1 x seq_len
        return F.softmax(attn_energies).unsqueeze(0).unsqueeze(0)
    
    def score(self, hidden, encoder_output):
        
        if self.method == 'dot':
            energy = hidden.dot(encoder_output)
            return energy
        
        elif self.method == 'general':
            energy = self.attn(encoder_output)
            energy = hidden.dot(energy)
            return energy
        
        elif self.method == 'concat':
            energy = self.attn(torch.cat((hidden, encoder_output), 1))
            energy = self.other.dot(energy)
            return energy

In [0]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, attn_model, hidden_size, output_size, n_layers=1, dropout_p=0.1):
        super(AttnDecoderRNN, self).__init__()

        # Keep parameters for reference
        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout_p = dropout_p

        # Define layers
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size * 2, hidden_size, n_layers, dropout=dropout_p)
        self.out = nn.Linear(hidden_size * 2, output_size)

        # Choose attention model
        if attn_model != 'none':
            self.attn = Attn(attn_model, hidden_size)

    def forward(self, word_input, last_context, last_hidden, encoder_outputs):
        # Note: we run this one step at a time

        # Get the embedding of the current input word (last output word)
        word_embedded = self.embedding(word_input).view(1, 1, -1) # S=1 x B x N

        # Combine embedded input word and last context, run through RNN
        rnn_input = torch.cat((word_embedded, last_context.unsqueeze(0)), 2)
        rnn_output, hidden = self.gru(rnn_input, last_hidden)

        # Calculate attention from current RNN state and all encoder outputs; apply to encoder outputs
        attn_weights = self.attn(rnn_output.squeeze(0), encoder_outputs)
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1)) # B x 1 x N

        # Final output layer (next word prediction) using the RNN hidden state and context vector
        rnn_output = rnn_output.squeeze(0) # S=1 x B x N -> B x N
        context = context.squeeze(1)       # B x S=1 x N -> B x N
        output = F.log_softmax(self.out(torch.cat((rnn_output, context), 1)))

        # Return final output, hidden state, and attention weights (for visualization)
        return output, context, hidden, attn_weights

In [0]:
#load_model
encoder = torch.load('encoder_general_100000.pth')
decoder = torch.load('decoder_general_100000.pth')

if USE_CUDA:
    encoder.cuda()
    decoder.cuda()

# Initialize optimizers and criterion
learning_rate = 0.0001
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
criterion = nn.NLLLoss()

In [19]:
random.choice(pairs)

['없 는 생 색 다 내 는 꼬 락 서 니 보 느 니 내 가 갈 고 만 다', '2']

In [0]:
def evaluate(sentence, max_length=MAX_LENGTH):
    input_variable = variable_from_sentence(input_lang, sentence)
    input_length = input_variable.size()[0]
    
    # Run through encoder
    encoder_hidden = encoder.init_hidden()
    encoder_outputs, encoder_hidden = encoder(input_variable, encoder_hidden)

    # Create starting vectors for decoder
    decoder_input = Variable(torch.LongTensor([[SOS_token]])) # SOS
    decoder_context = Variable(torch.zeros(1, decoder.hidden_size))
    if USE_CUDA:
        decoder_input = decoder_input.cuda()
        decoder_context = decoder_context.cuda()

    decoder_hidden = encoder_hidden
    
    decoded_words = []
    decoder_attentions = torch.zeros(max_length, max_length)
    
    # Run through decoder
    for di in range(max_length):
        decoder_output, decoder_context, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_context, decoder_hidden, encoder_outputs)
        decoder_attentions[di,:decoder_attention.size(2)] += decoder_attention.squeeze(0).squeeze(0).cpu().data

        # Choose top word from output
        topv, topi = decoder_output.data.topk(1)
        ni = topi[0][0]
        if ni == EOS_token:
#             decoded_words.append('<EOS>')
            break
        else:
            decoded_words.append(output_lang.index2word[ni])
            
        # Next input is chosen word
        decoder_input = Variable(torch.LongTensor([[ni]]))
        if USE_CUDA: decoder_input = decoder_input.cuda()
    
    return decoded_words, decoder_attentions[:di+1, :len(encoder_outputs)]

In [0]:
def evaluate_randomly():
    pair = random.choice(pairs)
    
    output_words, decoder_attn = evaluate(pair[0])
    output_sentence = ' '.join(output_words)
    
    print('>', pair[0])
    print('=', pair[1])
    print('<', output_sentence)
    print('')

In [24]:
output_words, decoder_attn = evaluate('의사의 꿈 키워 힘들게 공부하고 의대 진학해 의사 된 장군이.. 집안 형편도 어렵고 몸도 안 좋아 공부에도 한계가 있어 지방대 의대 감')
output_sentence = ' '.join(output_words)
print(output_sentence)

3


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:
evaluate_randomly()

In [93]:
# test_src_f = open('./non비속어_test_balanced.txt','r')
try:
    f = open('./result.txt','r')
    f.close()
except:
    test_src_f = open('./자가증식텍스트.txt','r')
    # test_targ_f = open('./data/data_50000/targ-test.txt','r')

    test_src = list()
    test_targ = list()
    test_pair = list()
    ##
    for line in src_f.read().splitlines():
        src_element = line.split('\t')[0].replace(" ", "")
        if len(src_element) > 20:
            src_element = src_element[len(src_element)-20:len(src_element)]
            src_element = list(src_element)
            src_element = ' '.join(src_element)
            src.append(src_element)
        else:
            src_element = list(src_element)
            src_element = ' '.join(src_element)
            src.append(src_element)
        targ.append(line.split('\t')[1])
        input_lang.index_words(src_element)
        output_lang.index_words(line.split('\t')[1])
    ##
    for line in test_src_f.read().splitlines():
        src_element = line.split('\t')[0].replace(" ", "")
        if len(src_element) > 20: 
          src_element = src_element[len(src_element)-20:len(src_element)]
          src_element = list(src_element)
          src_element = ' '.join(src_element)
          test_src.append(src_element)
        else:
          src_element = list(src_element)
          src_element = ' '.join(src_element)
          test_src.append(src_element)
        test_targ.append(line.split('\t')[1])


    for i in range(len(test_src)):
        test_pair.append([test_src[i]]+[test_targ[i]])

    result_f = open('./result.txt','w')
    result = list()
    for i in range(len(test_pair)):
        output_words, decoder_attn = evaluate(test_pair[i][0])
        output_sentence = ' '.join(output_words)
        result.append(output_sentence)
        print('%d : %s' %(i,output_sentence))
        result_f.write(output_sentence + '\n')
    result_f.close

/content/drive/Colab Notebooks/verify_ethics


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


0 : 2
1 : 2
2 : 1
3 : 1
4 : 3
5 : 3
6 : 3
7 : 2
8 : 1
9 : 2
10 : 1
11 : 1
12 : 3
13 : 2
14 : 3
15 : 2
16 : 3
17 : 1
18 : 1
19 : 3
20 : 3
21 : 2
22 : 3
23 : 1
24 : 2
25 : 3
26 : 2
27 : 1
28 : 1
29 : 3
30 : 3
31 : 3
32 : 3
33 : 1
34 : 3
35 : 3
36 : 2
37 : 2
38 : 3
39 : 2
40 : 3
41 : 1
42 : 1
43 : 2
44 : 1
45 : 2
46 : 3
47 : 3
48 : 3
49 : 3
50 : 1
51 : 3
52 : 3
53 : 3
54 : 3
55 : 1
56 : 1
57 : 2
58 : 3
59 : 3
60 : 1
61 : 2
62 : 3
63 : 3
64 : 3
65 : 3
66 : 2
67 : 3
68 : 2
69 : 3
70 : 3
71 : 3
72 : 1
73 : 1
74 : 3
75 : 1
76 : 1
77 : 1
78 : 3
79 : 1
80 : 3
81 : 3
82 : 1
83 : 3
84 : 1
85 : 3
86 : 3
87 : 3
88 : 2
89 : 2
90 : 3
91 : 1
92 : 3
93 : 2
94 : 3
95 : 3
96 : 1
97 : 3
98 : 3
99 : 3
100 : 1
101 : 3
102 : 1
103 : 1
104 : 3
105 : 1
106 : 1
107 : 3
108 : 1
109 : 3
110 : 3
111 : 1
112 : 3
113 : 3
114 : 3
115 : 1
116 : 2
117 : 2
118 : 1
119 : 3
120 : 2
121 : 3
122 : 2
123 : 3
124 : 1
125 : 2
126 : 3
127 : 2
128 : 3
129 : 3
130 : 3
131 : 1
132 : 3
133 : 1
134 : 3
135 : 2
136 : 1
137 : 3
138 : 

<function TextIOWrapper.close>

In [0]:
#get predict_seq from file
result = list()
f = open('result.txt')

for tag in f.read().splitlines():
  result.append(tag)

In [0]:
#창원
binary_result = list()
binary_result2 = list()

for element in result:
  if element == '1' or element == '2':
    element = '1'
  elif element == '3':
    element = '0'
  binary_result.append(element)
  
for element in result:
  if element == '1':
    element = '1'
  elif element == '2' or element == '3':
    element = '0'
  binary_result2.append(element)
  
predict_seq = binary_result2

In [95]:
for i in range(len(binary_result2)):
  binary_result2[i] = int(binary_result2[i])
print(get_score(binary_result2))

(0.6745562130177515, 0.6195652173913043, 0.7402597402597403, 0.8166666666666667)


In [0]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

In [10]:
origin_grams = list()
for sentence in origin_sentences:
    n_grams = set()
    for gram in range(len(sentence)-1):
        n_grams.add(sentence[gram:gram+2])
    origin_grams.append(n_grams)

    
generated_grams = list()
for sentence in sentences:
    n_grams = set()
    for gram in range(len(sentence)-1):
        n_grams.add(sentence[gram:gram+2])
    generated_grams.append(n_grams)
    
similarity_list = list()
for generated_gram in generated_grams:
    num_sim = 0
    count = 0
    for origin_gram in origin_grams:
        inter_set = generated_gram.intersection(origin_gram)
        if len(inter_set) > num_sim:
            num_sim = len(inter_set)
            sim_loc = count
        count += 1
    similarity_list.append((float(num_sim)/len(generated_gram),sim_loc))
print((similarity_list))

[(0.07692307692307693, 0), (0.12, 16), (0.078125, 19), (0.09090909090909091, 110), (0.10112359550561797, 59), (0.08823529411764706, 42), (0.06060606060606061, 92), (0.125, 30), (0.03333333333333333, 21), (0.030303030303030304, 30), (0.06896551724137931, 72), (0.058823529411764705, 0), (0.04081632653061224, 36), (0.05063291139240506, 43), (0.058823529411764705, 7), (0.07246376811594203, 37), (0.057971014492753624, 5), (0.06382978723404255, 30), (0.12, 13), (0.058823529411764705, 9), (0.10204081632653061, 30), (0.08620689655172414, 59), (0.11475409836065574, 48), (0.08571428571428572, 108), (0.1076923076923077, 82), (0.10344827586206896, 82), (0.1, 23), (0.0547945205479452, 46), (0.09090909090909091, 49), (0.06097560975609756, 53), (0.020833333333333332, 0), (0.17647058823529413, 17), (0.04938271604938271, 62), (0.07692307692307693, 68), (0.047058823529411764, 59), (0.05952380952380952, 16), (0.05747126436781609, 59), (0.1111111111111111, 48), (0.13333333333333333, 48), (0.06818181818181

In [82]:
final_list = list()

for i in range(len(predict_blacklist)):
  if predict_blacklist[i] == predict_seq[i]:
    final_list.append(predict_blacklist[i])
  else:
    final_list.append(origin_tag[similarity_list[i][1]])
print(len(final_list))
get_score(final_list)

300


(0.7120418848167539,
 0.5964912280701754,
 0.8831168831168831,
 0.8166666666666667)

In [69]:
k= 0.07
predict_sim = list()
for i in range(len(sentences)):
  if similarity_list[i][0] < k:
    predict_sim.append(0)
  else:
    predict_sim.append(origin_tag[similarity_list[i][1]])

print(get_score(predict_sim))
print(get_score(predict_sim2))

[0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0]
(0.45106382978723414, 0.33544303797468356, 0.6883116883116883, 0.57)
(0.40563380281690137, 0.258992

In [97]:
final_list = list()
for i in range(len(predict_blacklist)):
  if predict_blacklist[i] == predict_seq[i]:
    final_list.append(predict_blacklist[i])
  elif predict_blacklist[i] == predict_sim[i]:
    final_list.append(predict_blacklist[i])
  else:
    final_list.append(predict_seq[i])
#     final_list.append(origin_tag[similarity_list[i][1]])
print(len(final_list))
get_score(final_list)

300


(0.7337278106508877, 0.6739130434782609, 0.8051948051948052, 0.85)